
# Working with Spotify tracks
Getting information about Spotify tracks and running basic operations with tracks.

## Import statements

In [1]:
# Enable import from my_spotify.py. Based on https://stackoverflow.com/a/35273613/1899061.
# Putting my_spotify.py in the root directory (without this cell) used to work, but then suddenly stopped working.
import os
import sys
my_spotify_module_path = os.path.abspath(os.path.join('..'))
if my_spotify_module_path not in sys.path:
    sys.path.append(my_spotify_module_path)

In [6]:
from my_spotify.my_spotify import get_spotify_object, get_all_tracks_from_playlist
import pandas as pd

## Get the Spotify object

In [7]:
spot = get_spotify_object('env/.env')

## Specify a playlist to work with

In [8]:
THE_BEATLES_PLAYLIST_URL_1 = 'https://open.spotify.com/playlist/6ZmqDRJKJf3v3LzYZAaGGU?si=6b487ba5f9b54374' # 215 songs

## Get the tracks from the playlist

In [9]:
tracks = get_all_tracks_from_playlist(THE_BEATLES_PLAYLIST_URL_1, 'env/.env')

## Check what some items from the "encompassing" dictionary corresponding to a track are
This "encompassing" dictionary includes an "embedded" dictionary pertaining to the track itself, as well as some other metadata.

In [23]:
# type(tracks)                          # list

t = tracks[0]
# type(track)                           # dict

display(t.keys())
display(t['added_at'])
display(t['is_local'])
display(t['primary_color'])
display(t['video_thumbnail'])

display(t['track'])

dict_keys(['added_at', 'added_by', 'is_local', 'primary_color', 'track', 'video_thumbnail'])

'2020-01-15T04:20:38Z'

False

None

{'url': None}

{'album': {'album_type': 'album',
  'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/3WrFJ7ztbogyGnTHbHJFl2'},
    'href': 'https://api.spotify.com/v1/artists/3WrFJ7ztbogyGnTHbHJFl2',
    'id': '3WrFJ7ztbogyGnTHbHJFl2',
    'name': 'The Beatles',
    'type': 'artist',
    'uri': 'spotify:artist:3WrFJ7ztbogyGnTHbHJFl2'}],
  'available_markets': ['AD',
   'AE',
   'AG',
   'AL',
   'AM',
   'AO',
   'AR',
   'AT',
   'AU',
   'AZ',
   'BA',
   'BB',
   'BD',
   'BE',
   'BF',
   'BG',
   'BH',
   'BI',
   'BJ',
   'BN',
   'BO',
   'BR',
   'BS',
   'BT',
   'BW',
   'BY',
   'BZ',
   'CA',
   'CD',
   'CG',
   'CH',
   'CI',
   'CL',
   'CM',
   'CO',
   'CR',
   'CV',
   'CW',
   'CY',
   'CZ',
   'DE',
   'DJ',
   'DK',
   'DM',
   'DO',
   'DZ',
   'EC',
   'EE',
   'EG',
   'ES',
   'ET',
   'FI',
   'FJ',
   'FM',
   'FR',
   'GA',
   'GB',
   'GD',
   'GE',
   'GH',
   'GM',
   'GN',
   'GQ',
   'GR',
   'GT',
   'GW',
   'GY',
   'HK',
   'HN',
   'HR',


## Check what some items from the "embedded" dictionary corresponding to a track are

In [42]:
track = t['track']
# type(track)                           # dict

display(track.keys())

display(track['track'])
display(track['name'])
display(track['popularity'])
display(track['album'])
display(track['album']['name'])
display(track['duration_ms'])
display(track['artists'][0]['name'])
display(track['id'])
display(track['uri'])
display(track['href'])
display(track['episode'])               # False
display(track['track'])                 # True

## Get the audio features for a track
See [this](https://developer.spotify.com/documentation/web-api/reference/#/operations/get-several-audio-features) for an explanation of the meanings of all these parameters.

In [43]:
spot.audio_features(['spotify:track:3KiexfmhxHvG5IgAElmTkd'])

## Create a list of URIs of all tracks from the playlist

In [44]:
uri_list = [t['track']['uri'] for t in tracks]
uri_list[:5]

## Create a list of *(uri, title, popularity, duration)* tuples of all tracks from the playlist

In [45]:
tracks_data = [(t['track']['uri'],
                t['track']['name'].split(' - Remastered')[0].split(' / Remastered')[0],
                t['track']['album']['name'].split(' (Remastered)')[0],
                t['track']['popularity'],
                int(round(t['track']['duration_ms'] / 1000, 0))) for t in tracks]
display(len(tracks_data))
display(tracks_data[:])

## Create a list of audio features dictionaries of all tracks from the playlist
See [this](https://developer.spotify.com/documentation/web-api/reference/#/operations/get-several-audio-features) for an explanation of the meanings of all these parameters.

**Important:** `spot.audio_features()` will accept max 100 URIs!

In [46]:
offset = 0
runs, last_run = divmod(len(tracks), 100)                                                       # how many full runs, 100 tracks each
tracks_audio_features_dicts = []

if runs > 0:                                                                                    # all full runs, 100 tracks each
    for _ in range(runs):
        tracks_audio_features_dicts.extend(spot.audio_features(uri_list[offset:(offset+100)]))
        offset += 100
tracks_audio_features_dicts.extend(spot.audio_features(uri_list[offset:(offset+last_run)]))     # last run, generally < 100 tracks

# display(len(tracks_audio_features_dicts))
# display(tracks_audio_features_dicts)

## Create a list of *(key, mode, tempo, time_signature, valence, danceability, energy, loudness, acousticness, instrumentalness, liveness, speechiness)* tuples of all tracks from the playlist
See [this](https://developer.spotify.com/documentation/web-api/reference/#/operations/get-several-audio-features) for an explanation of the meanings of all these parameters.

Use `tracks_audio_features_dicts` list created above.

In [47]:
tracks_audio_features = [(t['key'],
                          t['mode'],
                          t['tempo'],
                          t['time_signature'],
                          t['valence'],
                          t['danceability'],
                          t['energy'],
                          t['loudness'],
                          t['acousticness'],
                          t['instrumentalness'],
                          t['liveness'],
                          t['speechiness']) for t in tracks_audio_features_dicts]

# display(len(tracks_audio_features))
# display(tracks_audio_features)

## Join the tuples from *tracks_data* and *tracks_audio_features* pairwise to create *tracks_data_and_audio_features*

In [48]:
tracks_data_and_audio_features = [(d + af) for d, af in zip(tracks_data, tracks_audio_features)]
# display(tracks_data_and_audio_features[:5])

## Bundle it all together in the *get_playlist_tracks_data()*, *get_playlist_tracks_audio_features()* and *get_playlist_tracks_df()* functions
Assumption: the *.env* file is already created as explained in *spotify_authentication.ipynb*, and its relative path is passed as an argument.

### *get_playlist_tracks_data()*

In [10]:
def get_playlist_tracks_data(playlist_id: str, env_file_path: str) -> list:
    tracks = get_all_tracks_from_playlist(playlist_id, env_file_path)
    tracks_data = [(t['track']['uri'],
                    t['track']['name'].split(' - Remastered')[0].split(' / Remastered')[0],
                    t['track']['album']['name'].split(' (Remastered)')[0],
                    t['track']['popularity'],
                    int(round(t['track']['duration_ms'] / 1000, 0))) for t in tracks]
    return tracks_data

In [11]:
# Test get_tracks(data)
display(get_playlist_tracks_data(THE_BEATLES_PLAYLIST_URL_1, 'env/.env'))

[('spotify:track:3KiexfmhxHvG5IgAElmTkd',
  'I Saw Her Standing There',
  'Please Please Me',
  69,
  174),
 ('spotify:track:40qXGg5nRbcWzcFb26KWkQ',
  'Misery',
  'Please Please Me',
  53,
  109),
 ('spotify:track:2baEFuU0gQon0hgVRioI1o',
  'Anna (Go To Him)',
  'Please Please Me',
  57,
  177),
 ('spotify:track:3JQWLa88R35d971o5bIImd',
  'Chains',
  'Please Please Me',
  48,
  145),
 ('spotify:track:5ZBeML7Lf3FMEVviTyvi8l',
  'Twist And Shout',
  'Please Please Me',
  79,
  155),
 ('spotify:track:4dessGxnKXmTbHPhVgqODq',
  "There's A Place",
  'Please Please Me',
  50,
  110),
 ('spotify:track:7fh53ta3vAOGJMQ4i5tCHe',
  'A Taste Of Honey',
  'Please Please Me',
  49,
  123),
 ('spotify:track:7Aobt67JnaF7qN8jCCKvHq',
  'Do You Want To Know A Secret',
  'Please Please Me',
  65,
  117),
 ('spotify:track:7JxGM1R32ZqfwZou3VtnTg', 'Boys', 'Please Please Me', 50, 146),
 ('spotify:track:7529Z6b1rlGZFFPWjHPeV5',
  'Ask Me Why',
  'Please Please Me',
  51,
  147),
 ('spotify:track:6EHuOufBeL6

### *get_playlist_tracks_audio_features()*

In [12]:
def get_playlist_tracks_audio_features(playlist_id: str, env_file_path: str) -> list:
    spot = get_spotify_object(env_file_path)
    tracks = get_all_tracks_from_playlist(playlist_id, env_file_path)
    uri_list = [t['track']['uri'] for t in tracks]

    offset = 0
    runs, last_run = divmod(len(tracks), 100)                                                       # how many full runs, 100 tracks each
    tracks_audio_features_dicts = []

    if runs > 0:                                                                                    # all full runs, 100 tracks each
        for _ in range(runs):
            tracks_audio_features_dicts.extend(spot.audio_features(uri_list[offset:(offset+100)]))
            offset += 100
    tracks_audio_features_dicts.extend(spot.audio_features(uri_list[offset:(offset+last_run)]))     # last run, < 100 tracks

    tracks_audio_features = [(t['key'],
                              t['mode'],
                              t['tempo'],
                              t['time_signature'],
                              t['valence'],
                              t['danceability'],
                              t['energy'],
                              t['loudness'],
                              t['acousticness'],
                              t['instrumentalness'],
                              t['liveness'],
                              t['speechiness']) for t in tracks_audio_features_dicts]

    return tracks_audio_features

In [13]:
# Test get_tracks_audio_features()
display(get_playlist_tracks_audio_features(THE_BEATLES_PLAYLIST_URL_1, 'env/.env'))

[(4, 1, 160.109, 4, 0.971, 0.491, 0.801, -9.835, 0.27, 0, 0.0665, 0.0361),
 (0, 1, 133.348, 4, 0.882, 0.591, 0.605, -10.925, 0.707, 0, 0.309, 0.0413),
 (2, 1, 109.286, 4, 0.835, 0.608, 0.565, -11.06, 0.635, 0, 0.0601, 0.0336),
 (10, 1, 129.884, 4, 0.929, 0.654, 0.561, -10.551, 0.608, 0, 0.129, 0.0304),
 (2,
  1,
  124.631,
  4,
  0.937,
  0.482,
  0.849,
  -9.198,
  0.641,
  7.74e-06,
  0.0414,
  0.0452),
 (4,
  1,
  140.928,
  4,
  0.927,
  0.455,
  0.582,
  -10.009,
  0.629,
  4.22e-06,
  0.172,
  0.0292),
 (1, 0, 101.408, 3, 0.412, 0.42, 0.372, -11.416, 0.698, 0, 0.104, 0.0327),
 (4, 1, 124.451, 4, 0.609, 0.673, 0.349, -12.414, 0.608, 0, 0.38, 0.0368),
 (4, 1, 142.445, 4, 0.822, 0.402, 0.86, -10.31, 0.607, 0, 0.736, 0.0504),
 (4, 1, 133.942, 4, 0.597, 0.605, 0.394, -11.33, 0.767, 0, 0.0967, 0.0378),
 (4, 1, 139.388, 4, 0.706, 0.527, 0.48, -9.61, 0.334, 0, 0.0702, 0.028),
 (4, 0, 112.421, 4, 0.879, 0.608, 0.494, -12.211, 0.778, 0, 0.0926, 0.0345),
 (2, 1, 134.435, 4, 0.77, 0.635, 0.6

### *get_playlist_tracks_df()*

In [14]:
def get_playlist_tracks_df(playlist_id: str, env_file_path: str) -> pd.DataFrame:
    COLUMNS = [
        'URI',
        'Title',
        'Album',
        'Popularity',
        'Duration',
        'Key',
        'Mode',
        'Tempo',
        'Time_signature',
        'Valence',
        'Danceability',
        'Energy',
        'Loudness',
        'Acousticness',
        'Instrumentalness',
        'Liveness',
        'Speechiness'
    ]

    tracks_data = get_playlist_tracks_data(playlist_id, env_file_path)
    tracks_audio_features = get_playlist_tracks_audio_features(playlist_id, env_file_path)
    tracks_data_and_audio_features = [(d + af) for d, af in zip(tracks_data, tracks_audio_features)]
    return pd.DataFrame(tracks_data_and_audio_features, columns=COLUMNS)


In [15]:
# Test get_playlist_tracks_df()
tracks_df = get_playlist_tracks_df(THE_BEATLES_PLAYLIST_URL_1, 'env/.env')
display(tracks_df)

,URI,Title,Album,Popularity,Duration,Key,Mode,Tempo,Time_signature,Valence,Danceability,Energy,Loudness,Acousticness,Instrumentalness,Liveness,Speechiness
0,spotify:track:3KiexfmhxHvG5IgAElmTkd,I Saw Her Standing There,Please Please Me,69,174,4,1,160.109,4,0.971,0.491,0.801,-9.835,0.270000,0.000000,0.0665,0.0361
1,spotify:track:40qXGg5nRbcWzcFb26KWkQ,Misery,Please Please Me,53,109,0,1,133.348,4,0.882,0.591,0.605,-10.925,0.707000,0.000000,0.3090,0.0413
2,spotify:track:2baEFuU0gQon0hgVRioI1o,Anna (Go To Him),Please Please Me,57,177,2,1,109.286,4,0.835,0.608,0.565,-11.060,0.635000,0.000000,0.0601,0.0336
3,spotify:track:3JQWLa88R35d971o5bIImd,Chains,Please Please Me,48,145,10,1,129.884,4,0.929,0.654,0.561,-10.551,0.608000,0.000000,0.1290,0.0304
4,spotify:track:5ZBeML7Lf3FMEVviTyvi8l,Twist And Shout,Please Please Me,79,155,2,1,124.631,4,0.937,0.482,0.849,-9.198,0.641000,0.000008,0.0414,0.0452
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210,spotify:track:48TNLcToLs8DWkdj5vYdiW,Rain,Past Masters (Vols. 1 & 2 / Remastered),49,181,0,1,106.933,4,0.541,0.234,0.693,-8.277,0.000028,0.899000,0.4420,0.0408
211,spotify:track:1tM9TBNPjieSOZ2d5VAQ1y,Lady Madonna,Past Masters (Vols. 1 & 2 / Remastered),38,138,2,1,109.110,4,0.780,0.631,0.748,-7.604,0.385000,0.308000,0.2050,0.0300
212,spotify:track:379hxtlY5LvbPQa5LL6dPo,The Inner Light,Past Masters (Vols. 1 & 2 / Remastered),39,156,8,1,101.545,3,0.387,0.304,0.543,-10.619,0.751000,0.014500,0.0764,0.0340
213,spotify:track:5xf0zdP4KCfshyVn02D3Ea,Kansas City / Hey-Hey-Hey-Hey - Medley,Beatles For Sale,45,158,7,1,131.360,4,0.924,0.588,0.724,-6.630,0.614000,0.000089,0.8770,0.0335


## Save the resulting dataframe

In [11]:
from pathlib import Path
data_dir = Path().cwd().parent / 'data'
data_dir.mkdir(parents=True, exist_ok=True)
# print(data_dir)
tracks_df.to_csv(data_dir / 'The Beatles songs dataset, Spotify 1.csv', index=False)

## Repeat the process for the Esher Demos playlist
This playlist is important because of some songs that cannot be found elsewhere.

In [16]:
THE_ESHER_DEMOS_PLAYLIST = 'https://open.spotify.com/playlist/1ACQGYUwU0mkFoIu9ac4Tu?si=bc8505d1cd064300'

tracks_df = get_playlist_tracks_df(THE_ESHER_DEMOS_PLAYLIST, 'env/.env')
display(tracks_df)

,URI,Title,Album,Popularity,Duration,Key,Mode,Tempo,Time_signature,Valence,Danceability,Energy,Loudness,Acousticness,Instrumentalness,Liveness,Speechiness
0,spotify:track:7KdX4AwTqPHmTe2OOIHjk0,Back In The U.S.S.R. - Esher Demo,Back In The U.S.S.R.,0,180,7,1,127.719,4,0.557,0.658,0.5300,-8.480,0.756,0.000000,0.1850,0.0845
1,spotify:track:0dPsctAum4J6B9Z2xTRz2A,Dear Prudence - Esher Demo,The Beatles,33,287,10,0,89.173,4,0.271,0.387,0.2660,-12.380,0.655,0.012900,0.1430,0.0302
2,spotify:track:29zK9NIS1AMTzsek0t3Oge,Glass Onion - Demo / Anthology 3 Version,Anthology 3,27,111,4,1,113.604,4,0.700,0.770,0.1570,-14.220,0.780,0.000000,0.4490,0.0670
3,spotify:track:6a6XoJzGeBIDy9pcDSrOgQ,Glass Onion - Esher Demo,The Beatles,29,116,4,1,113.622,4,0.616,0.697,0.2880,-9.951,0.689,0.000000,0.1270,0.0441
4,spotify:track:4xNXqEBx1VXuph56zyUzBg,"Ob-La-Di, Ob-La-Da - Esher Demo",The Beatles,34,191,8,1,118.730,4,0.549,0.760,0.6010,-7.883,0.403,0.000000,0.0859,0.0495
5,spotify:track:4Fq5tBgj3RwAhffVY5p1Nl,The Continuing Story Of Bungalow Bill - Esher ...,The Beatles,28,161,8,0,109.317,4,0.494,0.594,0.3570,-9.262,0.642,0.000000,0.1630,0.0397
6,spotify:track:5klBb3ttDYWHY1lp0Zsyiw,While My Guitar Gently Weeps - Esher Demo,While My Guitar Gently Weeps,0,161,1,0,82.396,4,0.360,0.503,0.3210,-11.827,0.835,0.000000,0.3530,0.0468
7,spotify:track:64hXk7WAGOxtcmWLcZNqVO,Happiness Is A Warm Gun - Anthology 3 Version,Anthology 3,31,135,5,1,128.430,3,0.337,0.679,0.1620,-12.774,0.891,0.000008,0.0718,0.0561
8,spotify:track:2TqrXEPZiEwiNgEBpX0uXV,Happiness Is A Warm Gun - Esher Demo,The Beatles,29,115,5,1,138.625,5,0.186,0.587,0.1800,-12.486,0.771,0.000024,0.1610,0.0424
9,spotify:track:3W4s5oTDnVTO0jYGkTKksZ,I'm So Tired - Esher Demo,The Beatles,30,191,11,1,152.361,4,0.420,0.473,0.2850,-11.598,0.756,0.000000,0.9250,0.1010


In [17]:
from pathlib import Path
data_dir = Path().cwd().parent / 'data'
data_dir.mkdir(parents=True, exist_ok=True)
# print(data_dir)
tracks_df.to_csv(data_dir / 'The Beatles songs - Esher Demos dataset, Spotify only.csv', index=False)